In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import catboost as cb
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

In [ ]:
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
data = pd.read_excel('prelast.xlsx',skiprows=2)
dat = data.drop(columns=['неделя'])
dat['Начало нед'] = pd.to_datetime(dat['Начало нед'].values)
dat = data.fillna(0)
val = dat.iloc[244:]
dat = dat.iloc[:244]
train, test = train_test_split(dat,
                test_size=0.2,
                train_size=0.8,
                shuffle=True)
train = train.drop(columns=['Начало нед'])
test = test.drop(columns=['Начало нед'])
tr = Pool(train.drop(columns='Продажи, рубли'),label=train['Продажи, рубли'].values)
ts = Pool(test.drop(columns='Продажи, рубли'),label=test['Продажи, рубли'].values)
model = CatBoostRegressor(iterations=50_000,
                            depth=5,
                            random_seed=5,
                            learning_rate=0.0001,
                            loss_function='RMSE',
                            eval_metric='R2',
                            grow_policy='Lossguide',
                            )
mode = model.fit(tr, eval_set=ts, use_best_model=True, verbose_eval=True)


In [ ]:
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

params1 = {'iterations': 100000,
'learning_rate': 0.08732908539257885,
 'depth': 6,
 'subsample': 0.0868766242583672,
 'colsample_bylevel': 0.8929339919825698,
 'min_data_in_leaf': 173,
 'grow_policy': 'SymmetricTree',
 'loss_function': 'RMSE',
 'l2_leaf_reg': 9}

params2 = {'iterations': 100000,
 'learning_rate': 0.09782174747745702,
 'depth': 12,
 'loss_function': 'MAE',
 'eval_metric': 'R2',
 'subsample': 0.5500556447340613,
 'colsample_bylevel': 0.8295198990606868,
 'grow_policy': 'Lossguide',
 'min_data_in_leaf': 50}


base_models = [
('catboost', CatBoostRegressor(**params1)),
('catboost2', CatBoostRegressor(**params2)),
]

X_train = train.drop(columns='Продажи, рубли')
y_train = train['Продажи, рубли'].values

X_test = test.drop(columns='Продажи, рубли')
y_test = test['Продажи, рубли'].values


stacking_model = StackingRegressor(estimators=base_models, final_estimator=LinearRegression())

stacking_model.fit(X_train, y_train)

predictions = stacking_model.predict(X_test)
